In [13]:
from google import genai
import os
from google.genai import types
api_key = os.environ.get("GEMINI_API_KEY")


 # Define the function declaration for the model
schedule_meeting_function = {
    "name": "schedule_meeting",
    "description": "Schedules a meeting with specified attendees at a given time and date.",
    "parameters": {
        "type": "object",
        "properties": {
            "attendees": {
                "type": "array",
                "items": {"type": "string"},
                "description": "List of people attending the meeting.",
            },
            "date": {
                "type": "string",
                "description": "Date of the meeting (e.g., '2024-07-29')",
            },
            "time": {
                "type": "string",
                "description": "Time of the meeting (e.g., '15:00')",
            },
            "topic": {
                "type": "string",
                "description": "The subject or topic of the meeting.",
            },
        },
        "required": ["attendees", "date", "time", "topic"],
    },
}

client = genai.Client(api_key=api_key)

def schedule_meeting(attendees, date, time, topic):
    print("schedule_meeting")
    return f"Meeting scheduled with {', '.join(attendees)} on {date} at {time} about {topic}."

tools = types.Tool(function_declarations=[schedule_meeting_function])
config = types.GenerateContentConfig(tools=[tools])

# Send request with function declarations
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Schedule a meeting with Bob and Alice for 03/14/2025 at 10:00 AM about the Q3 planning.",
    config=config,
)
# Check for a function call
if response.candidates[0].content.parts[0].function_call:
    function_call = response.candidates[0].content.parts[0].function_call
    print(f"Function to call: {function_call.name}")
    print(f"Arguments: {function_call.args}")
    #  In a real app, you would call your function here:
    result = schedule_meeting(**function_call.args)
    print(f"Function result: {result}")
else:
    print("No function call found in the response.")
    print(response.text)

Function to call: schedule_meeting
Arguments: {'topic': 'Q3 planning', 'date': '2025-03-14', 'attendees': ['Bob', 'Alice'], 'time': '10:00'}
schedule_meeting
Function result: Meeting scheduled with Bob, Alice on 2025-03-14 at 10:00 about Q3 planning.


In [18]:
from google import genai
import os
from google.genai import types
api_key = os.environ.get("GEMINI_API_KEY")


 # Define the function declaration for the model
schedule_meeting_function = {
    "name": "schedule_meeting",
    "description": "Schedules a meeting with specified attendees at a given time and date.",
    "parameters": {
        "type": "object",
        "properties": {
            "attendees": {
                "type": "array",
                "items": {"type": "string"},
                "description": "List of people attending the meeting.",
            },
            "date": {
                "type": "string",
                "description": "Date of the meeting (e.g., '2024-07-29')",
            },
            "time": {
                "type": "string",
                "description": "Time of the meeting (e.g., '15:00')",
            },
            "topic": {
                "type": "string",
                "description": "The subject or topic of the meeting.",
            },
        },
        "required": ["attendees", "date", "time", "topic"],
    },
}

client = genai.Client(api_key=api_key)

def schedule_meeting(attendees: list[str], date: str, time: str, topic: str) -> str:
# def schedule_meeting(attendees, date, time, topic):
    """
    Schedules a meeting and returns a formatted confirmation message.

    Args:
        attendees (list[str]): List of people attending the meeting.
        date (str): Date of the meeting in YYYY-MM-DD format.
        time (str): Time of the meeting in HH:MM format.
        topic (str): The subject or topic of the meeting.

    Returns:
        str: A human-readable confirmation message.
    """
    if not attendees:
        return "No attendees specified. Cannot schedule meeting."

    formatted_attendees = ", ".join(attendees)
    return (
        f"✅ Meeting successfully scheduled!\n"
        f"📅 Date: {date}\n"
        f"⏰ Time: {time}\n"
        f"👥 Attendees: {formatted_attendees}\n"
        f"📌 Topic: {topic}"
    )

config = {
    "tools": [schedule_meeting],
}

chat = client.chats.create(model="gemini-2.0-flash", config=config)
response = chat.send_message("Schedule a meeting with Bob and Alice for 03/14/2025 at 10:00 AM about the Q3 planning.")
response.text

'OK. I have scheduled a meeting with Bob and Alice for March 14, 2025 at 10:00 AM regarding Q3 planning.\n'

In [ ]:
from google import genai

# --- Tool Schemas ---
schedule_meeting_tool = {
    "name": "schedule_meeting",
    "description": "Schedules a calendar meeting with attendees, date, time, and topic.",
    "parameters": {
        "type": "object",
        "properties": {
            "attendees": {
                "type": "array",
                "items": {"type": "string"},
                "description": "List of attendees' email addresses",
            },
            "date": {
                "type": "string",
                "description": "Date of the meeting (e.g., '2025-04-09')",
            },
            "time": {
                "type": "string",
                "description": "Time of the meeting (e.g., '14:00')",
            },
            "topic": {
                "type": "string",
                "description": "Meeting topic or subject",
            },
        },
        "required": ["attendees", "date", "time", "topic"],
    },
}

send_gmail_tool = {
    "name": "send_gmail",
    "description": "Sends an email to a recipient with a subject and message body.",
    "parameters": {
        "type": "object",
        "properties": {
            "to": {
                "type": "string",
                "description": "Recipient email address",
            },
            "subject": {
                "type": "string",
                "description": "Email subject",
            },
            "body": {
                "type": "string",
                "description": "Email content",
            },
        },
        "required": ["to", "subject", "body"],
    },
}

get_contact_tool = {
    "name": "get_contact",
    "description": "Retrieves the email address of a contact by name.",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "Name of the contact to look up",
            },
        },
        "required": ["name"],
    },
}

# --- Implementation Functions with Type Annotations and Printouts ---

def schedule_meeting(attendees: list[str], date: str, time: str, topic: str) -> dict:
    """
    Schedule a meeting with specified attendees, date, time, and topic.

    Example:
        schedule_meeting(["alice@example.com", "bob@example.com"], "2025-04-09", "14:00", "Roadmap Discussion")

    Returns:
        dict: Confirmation of the scheduled meeting.
    """
    print("📅 Executing tool: schedule_meeting")
    return {
        "status": "Meeting scheduled",
        "attendees": attendees,
        "date": date,
        "time": time,
        "topic": topic,
    }

def send_gmail(to: str, subject: str, body: str) -> dict:
    """
    Send an email to a contact.

    Example:
        send_gmail("alice@example.com", "Meeting Agenda", "Please see the attached agenda.")

    Returns:
        dict: Confirmation of the email being sent.
    """
    print("📧 Executing tool: send_gmail")
    return {
        "status": "Email sent",
        "to": to,
        "subject": subject,
        "body": body,
    }

def get_contact(name: str) -> dict:
    """
    Look up a contact's email by name.

    Example:
        get_contact("Alice Johnson")

    Returns:
        dict: Contact name and mock email.
    """
    print("📇 Executing tool: get_contact")
    email = f"{name.lower().replace(' ', '_')}@example.com"
    print(email)
    return {
        "name": name,
        "email": email,
    }

# --- Chat Setup ---
client = genai.Client(api_key=api_key)

config = {
    "tools": [schedule_meeting, send_gmail, get_contact],
}

chat = client.chats.create(model="gemini-2.0-flash", config=config)

response = chat.send_message(
    # "Can you find the email address for Alice?"
    # "Find Alice's email then Schedule a meeting with Alice on April 9, 2025 at 2pm about the roadmap."
    # "Send an email to Alice saying ‘Hi, please see the agenda for our meeting tomorrow.’ Subject: Meeting Agenda"
)

print("\n🧠 Gemini Response:")
print(response.text)

📇 Executing tool: get_contact
alice@example.com
📧 Executing tool: send_gmail

🧠 Gemini Response:
OK. I've sent an email to Alice with the subject 'Meeting Agenda' and the body 'Hi, please see the agenda for our meeting tomorrow.'



In [23]:
from google import genai
from google.genai import types


power_disco_ball = {
    "name": "power_disco_ball",
    "description": "Powers the spinning disco ball.",
    "parameters": {
        "type": "object",
        "properties": {
            "power": {
                "type": "boolean",
                "description": "Whether to turn the disco ball on or off.",
            }
        },
        "required": ["power"],
    },
}

start_music = {
    "name": "start_music",
    "description": "Play some music matching the specified parameters.",
    "parameters": {
        "type": "object",
        "properties": {
            "energetic": {
                "type": "boolean",
                "description": "Whether the music is energetic or not.",
            },
            "loud": {
                "type": "boolean",
                "description": "Whether the music is loud or not.",
            },
        },
        "required": ["energetic", "loud"],
    },
}

dim_lights = {
    "name": "dim_lights",
    "description": "Dim the lights.",
    "parameters": {
        "type": "object",
        "properties": {
            "brightness": {
                "type": "number",
                "description": "The brightness of the lights, 0.0 is off, 1.0 is full.",
            }
        },
        "required": ["brightness"],
    },
}

# Actual implementation functions
def power_disco_ball_impl(power: bool) -> dict:
    """Powers the spinning disco ball.

    Args:
        power: Whether to turn the disco ball on or off.

    Returns:
        A status dictionary indicating the current state.
    """
    print("⛏️ Executing tool: power_disco_ball_impl")
    print({"status": f"Disco ball powered {'on' if power else 'off'}"})
    return {"status": f"Disco ball powered {'on' if power else 'off'}"}

def start_music_impl(energetic: bool, loud: bool) -> dict:
    """Play some music matching the specified parameters.

    Args:
        energetic: Whether the music is energetic or not.
        loud: Whether the music is loud or not.

    Returns:
        A dictionary containing the music settings.
    """
    music_type = "energetic" if energetic else "chill"
    volume = "loud" if loud else "quiet"
    print("⛏️ Executing tool: start_music_impl")
    print({"music_type": music_type, "volume": volume})
    return {"music_type": music_type, "volume": volume}

def dim_lights_impl(brightness: float) -> dict:
    """Dim the lights.

    Args:
        brightness: The brightness of the lights, 0.0 is off, 1.0 is full.

    Returns:
        A dictionary containing the new brightness setting.
    """
    print("⛏️ Executing tool: dim_lights_impl")
    return {"brightness": brightness}

config = {
    "tools": [power_disco_ball_impl, start_music_impl, dim_lights_impl],
}

chat = client.chats.create(model="gemini-2.0-flash", config=config)
response = chat.send_message("Do everything you need to this place into party!")
# response = chat.send_message("can you only turn the disco ball on")
# response = chat.send_message("start a chill music with quiet volume")
# response = chat.send_message("start a chill music with quiet volume and dim the lights to 50% brightness")

# print("\nExample 2: Automatic function calling")
print(response.text)



# I've turned on the disco ball, started playing loud and energetic music, and dimmed the lights to 50% brightness. Let's get this party started!

⛏️ Executing tool: power_disco_ball_impl
{'status': 'Disco ball powered on'}
⛏️ Executing tool: start_music_impl
{'music_type': 'energetic', 'volume': 'loud'}
⛏️ Executing tool: dim_lights_impl
OK, I've turned on the disco ball, started some energetic and loud music, and dimmed the lights to 50%. Let's get this party started!

